In [1]:
DATASET = "WikiTableQuestions"
QUESTION_TYPE = "answer"
prompts_path = f"/home/vivek/kunal/results/gpt4/remaining/{DATASET}_{QUESTION_TYPE}_final_rerun/qid_to_prompts.json"

In [2]:
import json
with open(prompts_path, "r") as f:
    prompts = json.load(f)

In [3]:
print(len(prompts))

205


In [4]:
import base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [5]:
import os
final_requests = []
qid_to_gold = {}

for qid, prompt in prompts.items():
    qid_to_gold[qid] = prompt['gold_ans']
    image_path = prompt['prompt'][1]
    
    if DATASET == "Hybrid_QA_MM":
        BASE_IMAGE_DIR = "/home/vivek/kunal/table-images-final/HybridQA_old"
        prompt["prompt"][1] = os.path.join(BASE_IMAGE_DIR,prompt["prompt"][1].split("/")[-1])
    
    image_encoding = encode_image(prompt['prompt'][1])
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt['prompt'][0]},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{image_encoding}",
                        "detail": "high"
                    },
                },
            ],
        }
    ]
    body = {
        "model": "gpt-4o",
        "messages": messages,
        "max_tokens": 4096,
        "temperature": 0.0,
    }
    
    final_request = {
        "custom_id": qid,
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": body
    }
    final_requests.append(final_request)

In [6]:
print(final_requests[0]['custom_id'])

WSQL-08851


In [7]:
print(final_requests[10]['body']['messages'][0]['content'][0]['text'])

You are given a table in the form of an image. In the table, some entities (mentioned in text form originally) have been replaced by images that represent them. Based upon the context of the table while using real-world knowledge, your task is to identify the entities corresponding to the images in the table and answer the question. You must perform this task in the following steps:

Step 1: Reason about what should be the answer to the question by identifying the relevant entities represented by images using the context of the table and the question. The reasoning should be detailed and should be based upon the context of the table and the question, using real-world knowledge for answering the question. IMPORTANT: You must explore any kind of reasoning -- numerical, logical, knowledge-based needed for disambiguating the entities and answering the question.
Step 2: Based upon the reasoning provided, provide the answer to the question.

You are also provided with some question-answer ex

In [8]:
# gpt4/remaining/{DATASET}_{QUESTION_TYPE}_final_rerun/qid_to_prompts.json"
from math import ceil
import os
try:
    os.mkdir(f"/home/vivek/kunal/results/gpt4/remaining/{DATASET}_{QUESTION_TYPE}_final_rerun/temp_files")
except:
    pass

TEMP_STORAGE_DIR = f"/home/vivek/kunal/results/gpt4/remaining/{DATASET}_{QUESTION_TYPE}_final_rerun/temp_files"
num_files = ceil(len(final_requests)/100)
for i in range(num_files):
    with open(f"{TEMP_STORAGE_DIR}/test_file_{i}.jsonl", "w") as f:
        for request in final_requests[i*100:min(len(final_requests), (i+1)*100)]:
            f.write(json.dumps(request) + "\n")

In [9]:
len(final_requests)

200

In [10]:
from openai import OpenAI
from tqdm import tqdm
client = OpenAI(api_key="") # CogComp key


In [11]:
print(TEMP_STORAGE_DIR)

/home/vivek/kunal/results/gpt4/remaining/WikiSQL_answer_final_rerun/temp_files


In [12]:
batch_input_file_ids = []

for i in tqdm(range(num_files)):
  path = f"{TEMP_STORAGE_DIR}/test_file_{i}.jsonl"
  batch_input_file = client.files.create(
      file=open(path, "rb"),
      purpose="batch"
    )
  batch_input_file_ids.append(batch_input_file)

100%|██████████| 2/2 [00:40<00:00, 20.03s/it]


In [13]:
# Sanity check
temp_question_ids = set()
temp_og_questionids = set(list(prompts.keys()))
for i in range(num_files):
    path = f"{TEMP_STORAGE_DIR}/test_file_{i}.jsonl"
    with open(path, "r") as f:
        for line in f:
            temp_question_ids.add(json.loads(line)['custom_id'])
print(temp_og_questionids == temp_question_ids)

True


In [14]:
# print(temp_og_questionids-temp_question_ids)
print(len(temp_question_ids))
print(len(temp_og_questionids))

200
200


In [15]:
batch_input_file_ids

[FileObject(id='file-MWMvAMvRSZivIUH3WY0W1jG5', bytes=54591335, created_at=1719756254, filename='test_file_0.jsonl', object='file', purpose='batch', status='processed', status_details=None),
 FileObject(id='file-zVTwVjsUf8oxGNmhgLNCaUyZ', bytes=49815267, created_at=1719756273, filename='test_file_1.jsonl', object='file', purpose='batch', status='processed', status_details=None)]

In [16]:
print(list(prompts.items())[0])

('WSQL-08851', {'prompt': ['You are given a table in the form of an image. In the table, some entities (mentioned in text form originally) have been replaced by images that represent them. Based upon the context of the table while using real-world knowledge, your task is to identify the entities corresponding to the images in the table and answer the question. You must perform this task in the following steps:\n\nStep 1: Reason about what should be the answer to the question by identifying the relevant entities represented by images using the context of the table and the question. The reasoning should be detailed and should be based upon the context of the table and the question, using real-world knowledge for answering the question. IMPORTANT: You must explore any kind of reasoning -- numerical, logical, knowledge-based needed for disambiguating the entities and answering the question.\nStep 2: Based upon the reasoning provided, provide the answer to the question.\n\nYou are also prov

In [17]:
print(TEMP_STORAGE_DIR)

/home/vivek/kunal/results/gpt4/remaining/WikiSQL_answer_final_rerun/temp_files


In [18]:
with open(f"{TEMP_STORAGE_DIR}/batch_input_file_ids.json", "w") as f:
    json.dump([file.to_dict() for file in batch_input_file_ids], f)

In [19]:
batch_input_file_ids

[FileObject(id='file-MWMvAMvRSZivIUH3WY0W1jG5', bytes=54591335, created_at=1719756254, filename='test_file_0.jsonl', object='file', purpose='batch', status='processed', status_details=None),
 FileObject(id='file-zVTwVjsUf8oxGNmhgLNCaUyZ', bytes=49815267, created_at=1719756273, filename='test_file_1.jsonl', object='file', purpose='batch', status='processed', status_details=None)]

In [20]:
print(list(prompts.values())[10]['prompt'][0])

You are given a table in the form of an image. In the table, some entities (mentioned in text form originally) have been replaced by images that represent them. Based upon the context of the table while using real-world knowledge, your task is to identify the entities corresponding to the images in the table and answer the question. You must perform this task in the following steps:

Step 1: Reason about what should be the answer to the question by identifying the relevant entities represented by images using the context of the table and the question. The reasoning should be detailed and should be based upon the context of the table and the question, using real-world knowledge for answering the question. IMPORTANT: You must explore any kind of reasoning -- numerical, logical, knowledge-based needed for disambiguating the entities and answering the question.
Step 2: Based upon the reasoning provided, provide the answer to the question.

You are also provided with some question-answer ex

In [21]:
len(prompts)

200

In [22]:
responses_store = []
for idx, batch_file in enumerate(batch_input_file_ids):
  response = client.batches.create(
      input_file_id=batch_file.id,
      endpoint="/v1/chat/completions",
      completion_window="24h",
      metadata={
        "description": f"{DATASET}_{QUESTION_TYPE}_{str(idx)}_final batch completion",
      }
  )
  responses_store.append(response)

In [23]:
batch_input_file_ids

[FileObject(id='file-MWMvAMvRSZivIUH3WY0W1jG5', bytes=54591335, created_at=1719756254, filename='test_file_0.jsonl', object='file', purpose='batch', status='processed', status_details=None),
 FileObject(id='file-zVTwVjsUf8oxGNmhgLNCaUyZ', bytes=49815267, created_at=1719756273, filename='test_file_1.jsonl', object='file', purpose='batch', status='processed', status_details=None)]

In [24]:
for response in responses_store:
    print(response)

Batch(id='batch_KXVIBKQal2lPqzirb2vv7ju8', completion_window='24h', created_at=1719756287, endpoint='/v1/chat/completions', input_file_id='file-MWMvAMvRSZivIUH3WY0W1jG5', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1719842687, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'WikiSQL_answer_0_final batch completion'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
Batch(id='batch_vnD1DTHEOjRh6utAgXhbQEDN', completion_window='24h', created_at=1719756287, endpoint='/v1/chat/completions', input_file_id='file-zVTwVjsUf8oxGNmhgLNCaUyZ', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1719842687, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'WikiSQL_answer_1_final ba

In [25]:
with open(f"{TEMP_STORAGE_DIR}/responses_store.json", "w") as f:
    json.dump([response.to_dict() for response in responses_store], f)

In [107]:
from time import sleep
from IPython.display import clear_output
res = client.batches.list(limit=5)

temp_check_input_files = []
for re in res:
    print(re)

Batch(id='batch_b9MNKnN2awHRyPfxN2eJJNOq', completion_window='24h', created_at=1719791531, endpoint='/v1/chat/completions', input_file_id='file-6VD63mfJsZ4i1rpiUgZ2r0cT', object='batch', status='failed', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=Errors(data=[BatchError(code='invalid_type', line=1, message='Invalid type: expected an object, but got an array instead.', param=None)], object='list'), expired_at=None, expires_at=1719877931, failed_at=1719791532, finalizing_at=None, in_progress_at=None, metadata={'description': 'cot_0'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))
Batch(id='batch_Xk1J7Q7Y7Iou0IpN5QgHnIDn', completion_window='24h', created_at=1719791247, endpoint='/v1/chat/completions', input_file_id='file-1pbCO4Z16DTwOcUBy1tUE64z', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1719791265, error_file_id=None, errors=None, expired_at=None, expires_at

RateLimitError: Error code: 429 - {'error': {'message': "You've exceeded the 100 request/min rate limit, please slow down and try again.", 'type': 'invalid_request_error', 'param': None, 'code': 'rate_limit_exceeded'}}

In [21]:
import json
DATASET = "FetaQA"
QUESTION_TYPE = "implicit"

TEMP_STORAGE_DIR = f"/home/vivek/kunal/results/gpt4/remaining/{DATASET}_{QUESTION_TYPE}_final_rerun/temp_files"
with open(f"{TEMP_STORAGE_DIR}/responses_store.json", "r") as f:
    responses_store = json.load(f)

In [22]:
from openai import OpenAI
from tqdm import tqdm
# client = OpenAI(api_key="")
client = OpenAI(api_key=) # CogComp key


In [23]:
print(responses_store)

[{'id': 'batch_q0fqxBmpVThhtrphRjY9bvKn', 'completion_window': '24h', 'created_at': 1719755975, 'endpoint': '/v1/chat/completions', 'input_file_id': 'file-ZsL3EVEebpfR9TNxe7V8ijIL', 'object': 'batch', 'status': 'validating', 'cancelled_at': None, 'cancelling_at': None, 'completed_at': None, 'error_file_id': None, 'errors': None, 'expired_at': None, 'expires_at': 1719842375, 'failed_at': None, 'finalizing_at': None, 'in_progress_at': None, 'metadata': {'description': 'FetaQA_implicit_0_final batch completion'}, 'output_file_id': None, 'request_counts': {'completed': 0, 'failed': 0, 'total': 0}}]


In [24]:
import os
OUTPUT_STORAGE_DIRECTORY = f"/home/vivek/kunal/results/gpt4/remaining/{DATASET}_{QUESTION_TYPE}_final_rerun/output_files"
os.makedirs(OUTPUT_STORAGE_DIRECTORY, exist_ok=True)
output_file_ids = []
# responses_ids = ['batch_DpT9wKqWvrcuzBRHV5i9quR1', 'batch_v6U0COQ1vRRtWhO1pm8bKD9a', 'batch_wDL9xAXgz9XqHiBGOA7qHtQr', 'batch_3SABPJdEThg8YhmAacYcrKmk', 'batch_3hPfF0YTkVGoQRCSeBZXqmWZ']
# responses_ids = ['batch_j9bogqKPs8B5Iqcq1370SG6H', 'batch_3a3KS1hAtGDmofzrBVgOWSdy', 'batch_dJco3EWLbaQrqGLAObiaOhZ7', 'batch_SlLVKts4qZNl8hzCVlB41Wql', 'batch_anDMohlpyefwpc7D5ltB0xmY']
# response_ids = ["file-LuJgkGZASaEvVEL3RVgZrEhb","file-sdYdKOITqZHHsmSiI65iVRg5","file-YRZkhQyED1A0am2d7wS4nMzR","file-s4AJn6HSFPErcccTrUniL5AG","file-1MaXFtfQoVfUEmnKzJP1qAz5",]
# responses_ids = ["batch_4UFsux3cMiEHhT8jbDQWQ9tR", "batch_xvaa1oVOtULAbG8wpoWHr29j", "batch_ADvWDqcdEG3WJSVCaCTcu6fd", "batch_OsEcsawQ9PwGLzIu5RR04Wwz", "batch_AfwcChiTRy05oublZAjOlMvY"]
# responses_ids = ['batch_rjvvv8wM1tF7BZHsydsdA6El', 'batch_5X7Yx9c3dJqZYK3r69v1E3CS', 'batch_wmOsp5WXuJ6Qmqig1BHMLpEB', 'batch_1mjx35ceh5eAvZQgvGbnt5CO', 'batch_qtMrpKrbvZrCv3aIQKDvg8vb']
for response in responses_store:
#     print("YAY")
# for response_id in responses_ids:
    # batch_id = response_id
    batch_id = response["id"]
    status = client.batches.retrieve(batch_id)
    output_file_id = status.output_file_id
    output_file_ids.append(status.to_dict())

with open(f"{OUTPUT_STORAGE_DIRECTORY}/output_file_ids.json", "w") as f:
    json.dump(output_file_ids, f)

In [25]:
print(OUTPUT_STORAGE_DIRECTORY)

/home/vivek/kunal/results/gpt4/remaining/FetaQA_implicit_final_rerun/output_files


In [26]:
output_file_ids

[{'id': 'batch_q0fqxBmpVThhtrphRjY9bvKn',
  'completion_window': '24h',
  'created_at': 1719755975,
  'endpoint': '/v1/chat/completions',
  'input_file_id': 'file-ZsL3EVEebpfR9TNxe7V8ijIL',
  'object': 'batch',
  'status': 'completed',
  'cancelled_at': None,
  'cancelling_at': None,
  'completed_at': 1719756048,
  'error_file_id': None,
  'errors': None,
  'expired_at': None,
  'expires_at': 1719842375,
  'failed_at': None,
  'finalizing_at': 1719756045,
  'in_progress_at': 1719755976,
  'metadata': {'description': 'FetaQA_implicit_0_final batch completion'},
  'output_file_id': 'file-Woh6lh4Fhw6kIt6jfMdoUwBw',
  'request_counts': {'completed': 90, 'failed': 0, 'total': 90}}]

In [27]:

for file_id in output_file_ids:
    file_id = file_id["output_file_id"]
    print(file_id)
    content = client.files.content(file_id)
    with open(f"{OUTPUT_STORAGE_DIRECTORY}/{file_id}.jsonl", "wb") as f:
        f.write(content.content)

file-Woh6lh4Fhw6kIt6jfMdoUwBw


In [28]:
print(DATASET, QUESTION_TYPE)


FetaQA implicit


In [29]:
with open(f"/home/vivek/kunal/results/gpt4/remaining/{DATASET}_{QUESTION_TYPE}_final_rerun/qid_to_prompts.json", "r") as f:
    qid_to_prompts = json.load(f)

In [30]:
print(list(qid_to_prompts.values())[0].keys())

dict_keys(['prompt', 'gold_ans'])


In [31]:
output_file_ids

[{'id': 'batch_q0fqxBmpVThhtrphRjY9bvKn',
  'completion_window': '24h',
  'created_at': 1719755975,
  'endpoint': '/v1/chat/completions',
  'input_file_id': 'file-ZsL3EVEebpfR9TNxe7V8ijIL',
  'object': 'batch',
  'status': 'completed',
  'cancelled_at': None,
  'cancelling_at': None,
  'completed_at': 1719756048,
  'error_file_id': None,
  'errors': None,
  'expired_at': None,
  'expires_at': 1719842375,
  'failed_at': None,
  'finalizing_at': 1719756045,
  'in_progress_at': 1719755976,
  'metadata': {'description': 'FetaQA_implicit_0_final batch completion'},
  'output_file_id': 'file-Woh6lh4Fhw6kIt6jfMdoUwBw',
  'request_counts': {'completed': 90, 'failed': 0, 'total': 90}}]

In [32]:
ques_id_to_output = {}
ques_id_to_gold = {}
for file_id in output_file_ids:
    file_id = file_id["output_file_id"]
    with open(f"{OUTPUT_STORAGE_DIRECTORY}/{file_id}.jsonl", "r") as f:
        for line in f:
            line = json.loads(line)
            ques_id = line['custom_id']
            output = line['response']['body']['choices'][0]['message']['content']
            ques_id_to_output[ques_id] = output
            ques_id_to_gold[ques_id] = qid_to_prompts[ques_id]['gold_ans']

In [33]:
len(ques_id_to_output)

90

In [35]:
with open(f"/home/vivek/kunal/results/gpt4/remaining/{DATASET}_{QUESTION_TYPE}_final_rerun/qid_to_output.json", "w") as f:
    json.dump(ques_id_to_output, f, indent=4)

In [36]:
with open(f"/home/vivek/kunal/results/gpt4/remaining/{DATASET}_{QUESTION_TYPE}_final_rerun/qid_to_gold.json", "w") as f:
    json.dump(ques_id_to_gold, f, indent=4)

In [37]:
# import json
# DATASET = "FetaQA"
# QUESTION_TYPE = "implicit"
with open(f"/home/vivek/kunal/results/gpt4/{DATASET}_{QUESTION_TYPE}/qid_to_output.json", "r") as f:
    ques_id_to_output_original = json.load(f)
with open(f"/home/vivek/kunal/results/gpt4/{DATASET}_{QUESTION_TYPE}/qid_to_gold.json", "r") as f:
    ques_id_to_gold_original = json.load(f)

In [38]:
import sys
from tqdm import tqdm
sys.path.append("/home/vivek/kunal/evaluation_metrics/")
from exact_match import EvaluationMetrics

final_ques_id_to_output = {}
final_ques_id_to_gold = {}

with open(f"/home/vivek/redone_samples_700cap/{DATASET}/test_{QUESTION_TYPE}_questions.jsonl", "r") as f:
    for line in f.readlines():
        line = json.loads(line)
        ques_id = int(line["question_id"])
        gold_ans = line["answer"]

        if DATASET == "FetaQA":
            ques_id = str(ques_id)
        
        if ques_id in ques_id_to_output_original:
            prediction = ques_id_to_output_original[ques_id]
        elif ques_id in ques_id_to_output:
            prediction = ques_id_to_output[ques_id]
        else:
            prediction = ""
        
        final_ques_id_to_output[ques_id] = prediction
        final_ques_id_to_gold[ques_id] = gold_ans


if DATASET == "FetaQA":
    ground_truths = []
    predictions = []

    for qid, response in tqdm(final_ques_id_to_output.items()):
        gold_ans = final_ques_id_to_gold[qid]
        # TODO: Fix the # after answer in the prompt
        # prompt = response["prompt"][0].split("You must follow the format of answers as demonstrated by the examples above. IMPORTANT: You must give the answer in the format 'Step 2:\n<answer>'.\n\n")[-1]
        # prompt = format_prompt(prompt)
        if type(gold_ans) == list:
            gold_ans = ", ".join([str(_) for _ in gold_ans])

        prediction = response

        prediction = prediction.split("Step 2:")[-1].strip()

        ground_truths.append(gold_ans)
        predictions.append(prediction)

    evaluator = EvaluationMetrics()
    bleu_score = evaluator.get_bleu_score(predictions, ground_truths)
    rouge_score = evaluator.get_rouge_score(predictions, ground_truths)
    # bleurt_score = evaluator.get_bleurt_score(predictions, ground_truths)
    print(len(ground_truths))

else:

    exact_match = 0
    substring_match = 0
    llm_match = 0
    incorrect = 0
    total = 0
    regex_match_cnt = 0
    f1_scores = []

    llm_evaluations = []
    evaluator = EvaluationMetrics()

    for qid, response in tqdm(final_ques_id_to_output.items()):
        gold_ans = final_ques_id_to_gold[qid]
        # TODO: Fix the # after answer in the prompt
        # prompt = response["prompt"][0].split("You must follow the format of answers as demonstrated by the examples above. IMPORTANT: You must give the answer in the format 'Step 2:\n<answer>'.\n\n")[-1]
        # prompt = format_prompt(prompt)
        if type(gold_ans) == list:
            gold_ans = ", ".join([str(_) for _ in gold_ans])

        prediction = response
        # print("-----------------")
        # print(gold_ans)
        # print(prediction)
        # print("-----------------")
        

        if prediction == "Error occurred.": #Temporarily ignoring error responses from gemini
            continue
        prediction = prediction.split("Step 2")[-1].strip()
        if prediction.startswith(":\n"):
            prediction = prediction[2:]

        if prediction.startswith("['"):
            prediction = prediction[2:-2]
        if len(prediction)>0 and prediction[0]=='[':
            prediction = prediction[1:-1]
        ##### GPT EVALUATOR PROMPT CONSTRUCTION #################################################
        # if c<100:
        #  c+=1
        #  gpt_evaluator_prompts[qid] = GPT_EVALUATOR_PROMPT_TEMPLATE.format(question = prompt,answer = gold_ans,candidate=prediction)
        #  print(gpt_evaluator_prompts[qid])

        if evaluator.regex_match(gold_ans, prediction):
            regex_match_cnt+=1

        if evaluator.compute_exact_match(gold_ans, prediction,DATASET):
            exact_match += 1
            substring_match += 1
            llm_match += 1
        else:
            if evaluator.gold_ans_in_prediction(prediction, gold_ans):
                substring_match += 1
                # print("----------------------")
                # print(gold_ans)
                # print(prediction)

            # llm_evaluations.append({
            #     "qid": qid,
            #     "gold_ans": gold_ans,
            #     "prediction": prediction,
            #     "question": test_questions[qid]["question"]
            # })
            else:
                pass
                # print("----------------")
                # print(prompt)
                # print("################")

                # print(gold_ans)
                # print("################")

                # print(prediction)
                # print("################")

                # print(response["prompt"][1])
                # print("----------------")

        f1_scores.append(evaluator.compute_f1_score(gold_ans, prediction))

    mean_f1_score = sum(f1_scores) / len(f1_scores)
    accuracy = exact_match / len(f1_scores)
    substring_accuracy = substring_match / len(f1_scores)
    regex_accuracy = regex_match_cnt / len(f1_scores)

    print(f"{DATASET} {QUESTION_TYPE} Results:")
    print(f"Exact match: {accuracy*100}")
    print(f"Substring match: {substring_accuracy*100}")
    print(f"Mean F1 score: {mean_f1_score}")
    print(f"Total: {len(f1_scores)}")



100%|██████████| 500/500 [00:00<00:00, 863380.82it/s]


====BLEU RESULTS====
{'score': 19.79457837553391, 'counts': [5539, 2811, 1696, 1083], 'totals': [12446, 11946, 11446, 10946], 'precisions': [44.504258396271894, 23.53088900050226, 14.817403459723922, 9.894025214690298], 'bp': 1.0, 'sys_len': 12446, 'ref_len': 9765}
=====ROUGE RESULTS=====
{'rouge1': 0.5529500498155062, 'rouge2': 0.33498984261270026, 'rougeL': 0.4606374391879325, 'rougeLsum': 0.46092885616595214}
500
